In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install Pillow
!pip install --upgrade azure-cognitiveservices-vision-face

In [ ]:
import io
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person

In [ ]:
#consts
dataset_path = "/content/gdrive/MyDrive/MIT_dataset/images/"
file_extension = ".jpg"
result_map = dict()
face_id_map = dict() 

# This key will serve all examples in this document.
KEY = "ed70949ab27b42c9a69347cfb9d38cf4"

# This endpoint will be used in all examples in this quickstart.
ENDPOINT = "https://ovidiu.cognitiveservices.azure.com/"

# Create an authenticated FaceClient.
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

In [ ]:
def init():
  global result_map, face_id_map
  for filename in sorted(os.listdir(dataset_path)):
    #print(filename)
    key = filename.split('.')[0] #get file names without extension

    print("getting face id for ", key, "...")
    complete_file_path = dataset_path + key + file_extension
    face_id = detect_image(complete_file_path)
    face_id_map.update({key : face_id}) #insert face id in map
    
    result_map.update({key : None}) #insert keys in map
  

In [ ]:
def detect_image(img_path, detection_model='detection_03'):
  detected_face = face_client.face.detect_with_stream(open((os.path.join(img_path)),'r+b'), detection_model=detection_model, return_face_id=True)
  
  if detected_face[0].face_id:
    return detected_face[0].face_id
  else:
    print("detect_image failed for: ", img_path)
    return None

In [ ]:
def compare_images(face_id_1, face_id_2):

  if not face_id_1 or not face_id_2:
    print("compare_images failed for invalid face_id_1=", face_id_1, "or/and invalid face_id_2=", face_id_2)
    return None

  verify_result_same = face_client.face.verify_face_to_face(face_id_1, face_id_2)

  if verify_result_same:
    return round(float(verify_result_same.confidence) * 100,2)
  else:
    print("compare_images failed for face_id_1= ", face_id_1, " and face_id_2= ", face_id_2)
    return None

In [ ]:
def add_results():
  global result_map, face_id_map

  for key1 in result_map.keys():

    person_confidence_score = dict()

    for key2 in face_id_map.keys():
    
      score = compare_images(face_id_map.get(key1), face_id_map.get(key2))
      print("key1:", key1, " key2:", key2, " score:", score)
      person_confidence_score.update({key2 : score})

    if person_confidence_score :
      result_map.update({key1 : person_confidence_score})

  #print(json.dumps(result_map, indent = 4))

In [ ]:
def show_results():
  print("========================================FINAL RESULTS========================================")
  for key, person_confidence_score_map in sorted (result_map.items()):
    score = None
    if person_confidence_score_map is not None:
      score = person_confidence_score_map.values()

    print(key, ":", score)

In [ ]:
init()
add_results()
show_results()

key1: 0_1  key2: 0_1  score: 100.0
key1: 0_1  key2: 0_2  score: 75.72
key1: 0_1  key2: 0_3  score: 43.52
key1: 0_1  key2: 0_4  score: 80.9
key1: 0_1  key2: 0_5  score: 39.42
key1: 0_1  key2: 1_1  score: 31.57
key1: 0_1  key2: 1_2  score: 19.95
key1: 0_1  key2: 1_3  score: 27.02
key1: 0_1  key2: 1_4  score: 19.46
key1: 0_1  key2: 1_5  score: 23.24
key1: 0_1  key2: 2_1  score: 8.56
key1: 0_1  key2: 2_2  score: 9.09
key1: 0_1  key2: 2_3  score: 8.18
key1: 0_1  key2: 2_4  score: 8.3
key1: 0_1  key2: 2_5  score: 8.17
key1: 0_1  key2: 2_6  score: 8.47
key1: 0_1  key2: 3_1  score: 9.39
key1: 0_1  key2: 3_2  score: 8.82
key1: 0_1  key2: 3_3  score: 8.5
key1: 0_1  key2: 3_4  score: 9.52
key1: 0_1  key2: 3_5  score: 8.82
key1: 0_1  key2: 4_1  score: 9.89
key1: 0_1  key2: 4_2  score: 9.86
key1: 0_1  key2: 4_3  score: 8.53
key1: 0_1  key2: 4_4  score: 10.45
key1: 0_1  key2: 4_5  score: 8.84
key1: 0_1  key2: 4_6  score: 9.67
key1: 0_1  key2: 5_1  score: 29.98
key1: 0_1  key2: 5_2  score: 14.69
key1

In [ ]:
compare_images(face_id_map.get(face_id_map.keys()[0]), face_id_map.get(face_id_map.keys()[1]))

TypeError: ignored